# <span style="color:blue"> **Case Study по модулю SQL** </span>
## <span style="color:blue"> **Испольнитель: Шероз Гуломалиев** </span>
## <span style="color:blue"> **10/03/2025** </span>

## <span style="color:green"> **Блок 1. Создание схемы и таблиц** </span>

### 1. В базе данных создайте новую схему adv_works.
### 2. На основе данных из файла создайте в схеме adv_works таблицы и соедините их между собою по основным и внешним ключам.

In [465]:
import plotly.express as px
import pandas as pd
import sqlalchemy as db
from sqlalchemy import create_engine, text

In [467]:
username = "postgres"  
password = "8888" 
host = "localhost"  
port = "5432"  
database = "postgres"  

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")


with engine.connect() as connection:
    connection.execute(text("create schema if not exists adv_works;"))
    connection.commit()



In [469]:
file_path = "adventure_works.xlsx"
xls = pd.ExcelFile(file_path)

def clean_column_names(df):
    df.columns = (
        df.columns
        .str.replace(r'([a-z])([A-Z])', r'\1_\2', regex=True)
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df


In [471]:
for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    df = clean_column_names(df)
    table_name = sheet_name.lower()
    df.to_sql(table_name, con=engine, schema="adv_works", if_exists="replace", index=False)
    print(f" Данные {sheet_name} загружены в adv_works.{table_name}")

print(" Все данные загружены успешно!")

 Данные Customers загружены в adv_works.customers
 Данные Products загружены в adv_works.products
 Данные Territory загружены в adv_works.territory
 Данные Sales загружены в adv_works.sales
 Данные ProductCategory загружены в adv_works.productcategory
 Данные ProductSubCategory загружены в adv_works.productsubcategory
 Все данные загружены успешно!


In [473]:
# Добавляем Primary key

primary_key_queries = [
    "alter table adv_works.customers add primary key (customer_key);",
    "alter table adv_works.sales add primary key (sales_order_number, sales_order_line_number);",
    "alter table adv_works.products add primary key (product_key);",
    "alter table adv_works.productsubcategory add primary key (product_subcategory_key);",
    "alter table adv_works.productcategory add primary key (product_category_key);",
    "alter table adv_works.territory add primary key (territory_key);"
]

with engine.connect() as connection:
    for query in primary_key_queries:
        connection.execute(text(query))
    connection.commit()
print("PRIMARY KEY добавлены!")

PRIMARY KEY добавлены!


In [477]:
# SQL-запросы для FOREIGN KEY
foreign_key_queries = [
    "alter table adv_works.sales add constraint fk_sales_customer foreign key (customer_key) references adv_works.customers(customer_key);",
    "alter table adv_works.sales add constraint fk_sales_product foreign key (product_key) references adv_works.products(product_key);",
    "alter table adv_works.sales add constraint fk_sales_territory foreign key (sales_territory_key) references adv_works.territory(territory_key);",
    "alter table adv_works.products add constraint fk_products_subcategory foreign key (product_subcategory_key) references adv_works.productsubcategory(product_subcategory_key);",
    "alter table adv_works.productsubcategory add constraint fk_subcategory_category foreign key (product_category_key) references adv_works.productcategory(product_category_key);"
]


with engine.connect() as connection:
    for query in foreign_key_queries:
        try:
            connection.execute(text(query))
            print(f" Добавлен FOREIGN KEY: {query}")
        except Exception as e:
            print(f"Ошибка при добавлении FOREIGN KEY: {e}")

    connection.commit()

print(" Все FOREIGN KEY успешно добавлены!")


 Добавлен FOREIGN KEY: alter table adv_works.sales add constraint fk_sales_customer foreign key (customer_key) references adv_works.customers(customer_key);
 Добавлен FOREIGN KEY: alter table adv_works.sales add constraint fk_sales_product foreign key (product_key) references adv_works.products(product_key);
 Добавлен FOREIGN KEY: alter table adv_works.sales add constraint fk_sales_territory foreign key (sales_territory_key) references adv_works.territory(territory_key);
 Добавлен FOREIGN KEY: alter table adv_works.products add constraint fk_products_subcategory foreign key (product_subcategory_key) references adv_works.productsubcategory(product_subcategory_key);
 Добавлен FOREIGN KEY: alter table adv_works.productsubcategory add constraint fk_subcategory_category foreign key (product_category_key) references adv_works.productcategory(product_category_key);
 Все FOREIGN KEY успешно добавлены!


## <span style="color:green"> **Блок 2. Аналитические задачи** </span>

### <span style="color:orange"> **Секция 1. Анализ клиентов** </span>


#### 1. Сегментация по доходу: Посчитайте средний годовой личный доход клиентов (YearlyIncome) в разбивке по роду деятельности (Occupation). Итоговая таблица должна содержать следующие поля: occupation, number_of_customers, avg_income.
#### 2. Семейный профиль: Посчитайте долю (в процентах) клиентов с детьми и долю клиентов без детей. Итоговая таблица должна содержать следующие поля: has_children (где 1 означает - имеет детей и 0 - не имеет детей), pct_of_customer_base.
#### 3. Высокодоходные клиенты: Сформируйте список топ 10 клиентов с наибольшей суммой покупок (поле SalesAmount). Итоговая таблица должна содержать следующие поля: customer_key, customer_name, total_purchase.
#### 4. Влияние семейного положения: Посчитайте среднюю сумму продаж в разбивке по семейному положению клиентов (MaritalStatus) и определите насколько сильно различаются средние суммы между двумя группами. Итоговая таблица должна содержать следующие поля: year, marital_status, avg_sales_amount.

In [479]:
queries = {
    "segmentation_by_income": """
        select 
            occupation,
            count(*) as number_of_customers,
            round(avg(yearly_income), 2) as avg_income
        from adv_works.customers
        group by occupation
        order by avg_income desc;
    """,
    
    "family_profile": """
        select 
            case 
                when number_children_at_home > 0 then 1
                else 0
            end as has_children,
            count(*) as number_of_customers,
            round(count(*) * 100.0 / (select count(*) from adv_works.customers), 2) as pct_of_customer_base
        from adv_works.customers
        group by has_children;
    """,
    
   "top_10_customers": """
        select 
            s.customer_key, 
            c.name as customer_name, 
            round(cast(sum(s.sales_amount) as numeric), 2) as total_purchase
        from adv_works.sales s
        left join adv_works.customers c on s.customer_key = c.customer_key
        group by s.customer_key, c.name
        order by total_purchase desc
        limit 10;
    """,
    "query_marital_sales" : """
        select 
            extract(year from s.order_date) as year, 
            c.marital_status, 
            round(avg(s.sales_amount)::numeric, 2) as avg_sales_amount
        from adv_works.sales s
        left join adv_works.customers c on s.customer_key = c.customer_key
        group by year, c.marital_status
        order by year, avg_sales_amount desc;
"""
    
}


for query_name, query in queries.items():
    df = pd.read_sql(query, engine)
    print(f"{query_name.replace('_', ' ').title()}")
    display(df)


Segmentation By Income


,occupation,number_of_customers,avg_income
0,Management,3075,92325.20
1,Professional,5520,74184.78
2,Skilled Manual,4577,51715.10
3,Clerical,2928,30710.38
4,Manual,2384,16451.34


Family Profile


,has_children,number_of_customers,pct_of_customer_base
0,0,11116,60.14
1,1,7368,39.86


Top 10 Customers


,customer_key,customer_name,total_purchase
0,12301,Nichole Nara,13295.38
1,12132,Kaitlyn Henderson,13294.27
2,12308,Margaret He,13269.27
3,12131,Randall Dominguez,13265.99
4,12300,Adriana Gonzalez,13242.70
5,12321,Rosa Hu,13215.65
6,12124,Brandi Gill,13195.64
7,12307,Brad She,13173.19
8,12296,Francisco Sara,13164.64
9,11433,Maurice Shan,12909.67


Query Marital Sales


,year,marital_status,avg_sales_amount
0,2001.0,M,3245.03
1,2001.0,S,3203.84
2,2002.0,S,2482.13
3,2002.0,M,2397.07
4,2003.0,S,427.78
5,2003.0,M,378.56
6,2004.0,S,318.05
7,2004.0,M,290.64


Выводы по секция №1:

1. Клиенты с более высокой оплатой труда (управленцы и профессионалы) имеют наибольший средний доход.
Ручные рабочие (Manual) зарабатывают в 5 раз меньше, чем менеджеры, что может указывать на разный уровень покупательной способности этих клиентов.
Skilled Manual (квалифицированные рабочие) зарабатывают больше, чем Clerical (офисные сотрудники), что говорит о высокой оплате труда технических специалистов.


2. Более 60% клиентов не имеют детей, что говорит о значительной доле одиноких или бездетных семейных покупателей.
39.86% клиентов имеют детей, что важно учитывать при разработке маркетинговых кампаний.



3. Средний объем покупок ТОП-10 клиентов превышает $13,000.
Клиенты с высокими расходами могут быть лояльными покупателями, если поддерживать с ними персональное взаимодействие.


4. В 2001-2002 годах продажи были значительно выше, чем в 2003-2004 годах.
Женатые клиенты в среднем тратят чуть больше, чем одинокие, но разница небольшая.
Снижение продаж с 2003 года говорит о кризисе или изменении предпочтений клиентов.
Рекомендации:



### <span style="color:orange"> **Секция 2. Анализ продаж** </span>

#### 1. Ежемесячные продажи: Создайте отчёт продаж по месяцам за последние 2 года (2003, 2004). Итоговая таблица должна содержать следующие поля: year, monthkey, month_name, sales_count (количество продаж), sales_amount.
#### 2. Продажи по регионам: Посчитайте сумму продаж в разбивке по регионам. Итоговая таблица должна содержать следующие поля: region, sales_count, sales_amount.

In [481]:
queries = {
    "monthly_sales": """
        select 
            extract(year from s.order_date) as year,
            extract(month from s.order_date) as monthkey,
            to_char(s.order_date, 'Month') as month_name,
            count(s.sales_order_number) as sales_count,
            round(sum(s.sales_amount)::numeric, 2) as sales_amount
        from adv_works.sales s
        where extract(year from s.order_date) in (2003, 2004)
        group by year, monthkey, month_name
        order by year, monthkey;
    """,
    "region_sales": """
        select 
            t.region,
            count(s.sales_order_number) AS sales_count,
            round(sum(s.sales_amount)::numeric, 2) as sales_amount
        from adv_works.sales s
        left join adv_works.territory t on s.sales_territory_key = t.territory_key
        group by t.region
        order by sales_amount desc; """
  
}

for query_name, query in queries.items():
    df = pd.read_sql(query, engine)
    print(f" {query_name.replace('_', ' ').title()}")
    display(df)


 Monthly Sales


,year,monthkey,month_name,sales_count,sales_amount
0,2003.0,1.0,January,244,438865.17
1,2003.0,2.0,February,272,489090.34
2,2003.0,3.0,March,272,485574.79
3,2003.0,4.0,April,294,506399.27
4,2003.0,5.0,May,335,562772.56
5,2003.0,6.0,June,321,554799.23
6,2003.0,7.0,July,1411,886668.84
7,2003.0,8.0,August,3819,847413.51
8,2003.0,9.0,September,3885,1010258.13
9,2003.0,10.0,October,4146,1080449.58


 Region Sales


,region,sales_count,sales_amount
0,Australia,13345,9061000.58
1,Southwest,12265,5718150.81
2,Northwest,8993,3649866.55
3,United Kingdom,6906,3391712.21
4,Germany,5625,2894312.34
5,France,5558,2644017.71
6,Canada,7620,1977844.86
7,Southeast,39,12238.85
8,Northeast,27,6532.47
9,Central,20,3000.83


Секция 2
1. Выводы по ежемесячным продажам:
Сильный рост продаж в декабре каждого года – самая прибыльная пора.
Летний сезон (июль-август) показывает высокий спрос, но в июле 2004 года он упал до $50,840.
Продажи стабильно растут с января по июнь 2004 года, достигая пика в июне.
Заметный скачок продаж в последние 3 месяца года (октябрь–декабрь), возможно, из-за сезонных скидок и праздничных покупок.

2. Выводы по регионам:
Австралия – самый прибыльный регион ($9 млн), почти в два раза больше, чем следующий регион (Southwest – $5,7 млн).
Юго-запад (Southwest) – второй крупнейший рынок, что может указывать на активный бизнес-центр или потребительский спрос.
Европейские страны (UK, Германия, Франция) демонстрируют стабильный спрос, но отстают от Австралии и США.
Слабые продажи в Центральном, Северо-Восточном и Юго-Восточном регионах, что указывает на низкий потенциал или слабый маркетинг в этих областях.



### <span style="color:orange"> **Секция 3. Анализ продуктов** </span>

#### 1. Доля продаж: Посчитайте какую долю от общих продаж составляет каждая категория продуктов. Итоговая таблица должна содержать следующие поля: year, product_key, product_category_key, english_product_category_name, sales_amount, pct_of_total_sales.
#### 2. Самые продаваемые продукты: Определите топ 5 продуктов с наибольшей суммой продаж. Итоговая таблица должна содержать следующие поля: product_key, product_name, english_product_category_name, sales_amount
#### 3. Маржа от продаж: Посчитайте разницу между суммой продаж (SalesAmount) за минусом себестоимости (TotalProductCost), налогов (*TaxAmt) и расходов на доставку (Freight) по каждому продукту в разбивке по годам и месяцам. Итоговая таблица должна содержать следующие поля: year, monthkey, month_name, product_key, product_name, sales_amount, total_product_cost, tax_amt, freight, margin, margin_pct (маржа как процент от суммы продаж).

In [483]:
queries = {
    "product_sales_share": """
        select 
            extract(year from s.order_date) as year,
            s.product_key,
            pc.product_category_key,
            pc.english_product_category_name,
            round(sum(s.sales_amount)::numeric, 2) as sales_amount,
            round((sum(s.sales_amount) * 100 / (select sum(sales_amount) from adv_works.sales))::numeric, 2) as pct_of_total_sales
        from adv_works.sales s
        inner join adv_works.products p on s.product_key = p.product_key
        inner join adv_works.productsubcategory psc on p.product_subcategory_key = psc.product_subcategory_key
        inner join adv_works.productcategory pc on psc.product_category_key = pc.product_category_key
        group by year, s.product_key, pc.product_category_key, pc.english_product_category_name
        order by pct_of_total_sales desc;
    """,
    
    "top_5_products": """
        select 
            s.product_key,
            p.product_name,
            pc.english_product_category_name,
            round(sum(s.sales_amount)::numeric, 2) as sales_amount
        from adv_works.sales s
        inner join adv_works.products p on s.product_key = p.product_key
        inner join adv_works.productsubcategory psc on p.product_subcategory_key = psc.product_subcategory_key
        inner join adv_works.productcategory pc on psc.product_category_key = pc.product_category_key
        group by s.product_key, p.product_name, pc.english_product_category_name
        order by sales_amount desc
        limit 5;
    """,

    "sales_margin": """
        select
            extract(year from s.order_date) as year,
            extract(month from s.order_date) as monthkey,
            to_char(s.order_date, 'month') as month_name,
            s.product_key,
            p.product_name,
            round(sum(s.sales_amount)::numeric, 2) as sales_amount,
            round(sum(s.total_product_cost)::numeric, 2) as total_product_cost,
            round(sum(s.tax_amt)::numeric, 2) as tax_amt,
            round(sum(s.freight)::numeric, 2) as freight,
            round(sum(s.sales_amount - s.total_product_cost - s.tax_amt - s.freight)::numeric, 2) as margin,
            round(
                (sum(s.sales_amount - s.total_product_cost - s.tax_amt - s.freight) * 100 /
                 nullif(sum(s.sales_amount), 0))::numeric, 2
            ) as margin_pct
        from adv_works.sales s
        inner join adv_works.products p on s.product_key = p.product_key
        group by year, monthkey, month_name, s.product_key, p.product_name
        order by year, monthkey, margin desc;
    """
}

for query_name, query in queries.items():
    df = pd.read_sql(query, engine)
    print(f"{query_name.replace('_', ' ').title()}")
    display(df)


Product Sales Share


,year,product_key,product_category_key,english_product_category_name,sales_amount,pct_of_total_sales
0,2002.0,312,1,Bikes,658401.68,2.24
1,2002.0,310,1,Bikes,608305.90,2.07
2,2002.0,313,1,Bikes,608305.90,2.07
3,2001.0,310,1,Bikes,593992.82,2.02
4,2004.0,353,1,Bikes,589277.46,2.01
...,...,...,...,...,...,...
311,2004.0,480,4,Accessories,4202.15,0.01
312,2003.0,482,3,Clothing,1024.86,0.00
313,2004.0,482,3,Clothing,1402.44,0.00
314,2003.0,481,3,Clothing,1204.66,0.00


Top 5 Products


,product_key,product_name,english_product_category_name,sales_amount
0,312,"Road-150 Red, 48",Bikes,1205876.99
1,310,"Road-150 Red, 62",Bikes,1202298.72
2,313,"Road-150 Red, 52",Bikes,1080637.54
3,314,"Road-150 Red, 56",Bikes,1055589.65
4,311,"Road-150 Red, 44",Bikes,1005493.87


Sales Margin


,year,monthkey,month_name,product_key,product_name,sales_amount,total_product_cost,tax_amt,freight,margin,margin_pct
0,2001.0,7.0,july,312,"Road-150 Red, 48",100191.56,60796.24,8015.32,2504.79,28875.21,28.82
1,2001.0,7.0,july,311,"Road-150 Red, 44",82300.21,49939.77,6584.02,2057.51,23718.92,28.82
2,2001.0,7.0,july,310,"Road-150 Red, 62",78721.94,47768.47,6297.76,1968.05,22687.66,28.82
3,2001.0,7.0,july,314,"Road-150 Red, 56",53674.05,32569.41,4293.92,1341.85,15468.86,28.82
4,2001.0,7.0,july,313,"Road-150 Red, 52",42939.24,26055.53,3435.14,1073.48,12375.09,28.82
...,...,...,...,...,...,...,...,...,...,...,...
1895,2004.0,7.0,july,481,"Racing Socks, M",152.83,57.16,12.23,3.82,79.62,52.10
1896,2004.0,7.0,july,488,"Short-Sleeve Classic Jersey, S",593.89,457.30,47.51,14.85,74.24,12.50
1897,2004.0,7.0,july,490,"Short-Sleeve Classic Jersey, L",431.92,332.58,34.55,10.80,53.99,12.50
1898,2004.0,7.0,july,479,Road Bottle Cage,89.90,33.62,7.19,2.25,46.84,52.10


Секция 3.
1. Выводы по доле продаж:
Категория "Bikes" (велосипеды) – абсолютный лидер, занимая наибольшую долю продаж.
Топовые модели Road-150 Red стабильно продаются с большим отрывом от других категорий.
Другие категории, такие как "Clothing" (одежда) и "Accessories" (аксессуары), имеют незначительный вклад в общие продажи.

2. Выводы по самым продаваемым товарам:
Все топовые товары относятся к линейке "Road-150 Red", подтверждая доминирующую роль велосипедов в продажах.
Продажи каждого из топ-5 велосипедов превышают $1 млн, что говорит о высоком спросе.
Отсутствие других категорий товаров в топ-5 может говорить о слабом маркетинге или низком интересе покупателей.

3. Выводы по марже:
Средняя маржа по категории "Bikes" составляет около 28-29%, что является хорошим показателем прибыльности.
Наибольшая маржа у моделей Road-150 Red, что подтверждает их высокую ценность для бизнеса.
Некоторые товары из категории "Clothing" (одежда) и "Accessories" (аксессуары) имеют более высокую маржинальность (52.10%), но их общий вклад в продажи незначителен.


### <span style="color:orange"> **Секция 4. Анализ трендов** </span>

#### 1. Квартальный рост: Посчитайте сумму продаж за каждый квартал и их процентное изменение по топ 2 наиболее продаваемым категориям. Итоговая таблица должна содержать следующие поля: year, quarter_id, product_category_key, english_product_category_name, quarter_sales_amount, quarter_over_quarter_growth_pct.
#### 2. Сравнение будних и выходных (суббота, воскресенье) дней: Посчитайте продажи в разбивке по годам и дням недели. Определите в какие дни в среднем сумма продаж больше. Определите является ли сумма продаж больше в будние или выходные дни. Итоговая таблица должна содержать следующие поля: year, day_name, is_weekend (где 1 означает выходной а 0 будний день), sales_amount.


In [485]:
queries = {
    "quarterly_growth": """
        with top_categories as (
            select 
                pc.product_category_key,
                pc.english_product_category_name,
                sum(s.sales_amount) as total_sales
            from adv_works.sales s
            inner join adv_works.products p on s.product_key = p.product_key
            inner join adv_works.productsubcategory psc on p.product_subcategory_key = psc.product_subcategory_key
            inner join adv_works.productcategory pc on psc.product_category_key = pc.product_category_key
            group by pc.product_category_key, pc.english_product_category_name
            order by total_sales desc
            limit 2
        ),
        sales_by_quarter as (
            select 
                extract(year from s.order_date) as year,
                extract(quarter from s.order_date) as quarter_id,
                pc.product_category_key,
                pc.english_product_category_name,
                sum(s.sales_amount) as quarter_sales_amount
            from adv_works.sales s
            inner join adv_works.products p on s.product_key = p.product_key
            inner join adv_works.productsubcategory psc on p.product_subcategory_key = psc.product_subcategory_key
            inner join adv_works.productcategory pc on psc.product_category_key = pc.product_category_key
            where pc.product_category_key in (select product_category_key from top_categories)
            group by year, quarter_id, pc.product_category_key, pc.english_product_category_name
        )
        select 
            s.year,
            s.quarter_id,
            s.product_category_key,
            s.english_product_category_name,
            s.quarter_sales_amount,
            round(
                (cast(s.quarter_sales_amount as numeric) - cast(lag(s.quarter_sales_amount) over (partition by s.product_category_key order by s.year, s.quarter_id) as numeric)) * 100 /
                nullif(cast(lag(s.quarter_sales_amount) over (partition by s.product_category_key order by s.year, s.quarter_id) as numeric), 0), 2
            ) as quarter_over_quarter_growth_pct
        from sales_by_quarter s
        order by s.year, s.quarter_id, s.product_category_key;
    """,

    "weekday_vs_weekend": """
        select 
            extract(year from s.order_date) as year,
            to_char(s.order_date, 'day') as day_name,
            case
                when extract(dow from s.order_date) in (0, 6) then 1 
                else 0
            end as is_weekend,
            round(sum(s.sales_amount)::numeric, 2) as sales_amount
        from adv_works.sales s
        group by year, day_name, is_weekend
        order by year, is_weekend desc, sales_amount desc;
    """
}


for query_name, query in queries.items():
    df = pd.read_sql(query, engine)
    print(f"{query_name.replace('_', ' ').title()}")
    display(df)


Quarterly Growth


,year,quarter_id,product_category_key,english_product_category_name,quarter_sales_amount,quarter_over_quarter_growth_pct
0,2001.0,3.0,1,Bikes,1.453523e+06,NaN
1,2001.0,4.0,1,Bikes,1.812851e+06,24.72
2,2002.0,1.0,1,Bikes,1.791698e+06,-1.17
3,2002.0,2.0,1,Bikes,2.014012e+06,12.41
4,2002.0,3.0,1,Bikes,1.396834e+06,-30.64
5,2002.0,4.0,1,Bikes,1.327799e+06,-4.94
6,2003.0,1.0,1,Bikes,1.413530e+06,6.46
7,2003.0,2.0,1,Bikes,1.623971e+06,14.89
8,2003.0,3.0,1,Bikes,2.569678e+06,58.23
9,2003.0,3.0,4,Accessories,1.186745e+05,NaN


Weekday Vs Weekend


,year,day_name,is_weekend,sales_amount
0,2001.0,sunday,1,526026.82
1,2001.0,saturday,1,505234.58
2,2001.0,friday,0,468724.50
3,2001.0,thursday,0,450281.09
4,2001.0,monday,0,447197.10
5,2001.0,wednesday,0,435300.15
6,2001.0,tuesday,0,433609.42
7,2002.0,sunday,1,937525.27
8,2002.0,saturday,1,915346.19
9,2002.0,wednesday,0,1004132.60


Секция 4.
1. Выводы по квартальному росту продаж:
Продажи велосипедов ("Bikes") показали сильный рост в 2003 и 2004 годах.
В 2003 году 3-й квартал (+58.23%) и 4-й квартал (+46.01%) были рекордными.
В 2004 году продажи продолжили расти, особенно во 2-м квартале (+27.69%).
Аксессуары ("Accessories") показали резкий рост в 2003 году (3-й и 4-й кварталы), но затем в 3-м квартале 2004 года обвалились на -83.11%.
Велосипеды остаются основной движущей силой продаж, их рост более стабилен, чем у аксессуаров.

2. Выводы по будним и выходным дням:
Продажи в будние дни практически равны продажам в выходные дни.
Например, в 2004 году в субботу ($1.48M) и в пятницу ($1.46M) продажи почти идентичны.
Пятница является самым доходным днем среди будней.
Суббота и воскресенье также демонстрируют высокий уровень продаж, особенно в 2003-2004 годах.
